In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as tt

if tt.cuda.is_available():
    print(tt.cuda.current_device())
    print(tt.cuda.device_count())
    print(tt.cuda.get_device_name(0))
    # device = tt.device("cuda")

0
1
NVIDIA GeForce GTX 1660 Ti


In [58]:
x_train = pd.read_csv("csv/x_train.csv")
y_train = pd.read_csv("csv/y_train.csv")
x_test = pd.read_csv("csv/x_test.csv")

In [59]:
ret_vol_cols = ["RET_"+str(i) for i in range(1,21)]+["VOLUME_"+str(i) for i in range(1,21)]
t = x_train[ret_vol_cols]
print(t.shape[0] - t.dropna().shape[0])

104435


In [60]:
# Torch sample code

class Model(tt.nn.Module):
    def __init__(self, inp_dim, out_dim):
        super(Model, self).__init__()
        self.linear = tt.nn.Linear(inp_dim, out_dim)
    def forward(self, x):
        print(type(x))
        y = tt.sigmoid(self.linear(x))
        return y

epochs = 2000
lr = 0.01
x = x_train[ret_vol_cols]
x = x.fillna(0)
model = Model(x.shape[0],1)
# model.to(device)
criterion = tt.nn.BCELoss()
optimizer = tt.optim.SGD(model.parameters(), lr=lr)

for i in range(epochs):
    x = tt.from_numpy(x)
    labels = tt.from_numpy(y_train.RET)
    optimizer.zero_grad()
    out = model(x)
    loss = criterion(tt.squeeze(out),labels)
    loss.backward()
    optimizer.step()

In [61]:
# duplicate stock on different date id's -> looking for trends.

stock = 133
df = x_train.loc[x_train["STOCK"] == stock][ret_vol_cols + ["INDUSTRY","INDUSTRY_GROUP","SECTOR","SUB_INDUSTRY","STOCK","DATE"]]
print(df.shape)

def f(data):
    print(data.axes)
    date_id = data[-1]
    data = data[:-6]
    rets,vols = data[:20],data[20:40]
    industry, industry_grp, sector, sub_industry = data[-6],data[-5],data[-4],data[-3]
    return date_id, rets, vols, industry, industry_grp, sector, sub_industry

for i in range(df.shape[0]):
    a = df.iloc[i]
    a_date_id, a_rets, a_vols, a_industry, a_industry_grp, a_sector, a_sub_industry = f(a)
    a_rets = a_rets.values
    for j in range(i+1,df.shape[0]):
        b = df.iloc[j]
        b_date_id, b_rets, b_vols, b_industry, b_industry_grp, b_sector, b_sub_industry = f(b)
        b_rets = b_rets.values
        # look for duplicates
        for k,b in enumerate(b_rets):
            if b != 0 and b in a_rets and a_date_id != b_date_id:
                t = a_rets.tolist()
                t = t[t.index(b):]
                if len(t) > 0 and len(b_rets[i:]) > 0:
                    print(f"{b_rets[k:]}\n{t}")
                    print(f"{a_industry} - {b_industry}\n{a_industry_grp} - {b_industry_grp}\n{a_sector} - {b_sector}\n{a_sub_industry} - {b_sub_industry}\n")

In [62]:
# Darts Transformer w TimeSeries data

melt_try = x_train
keep_vars = ['DATE', "STOCK", "INDUSTRY", "INDUSTRY_GROUP", "SECTOR", "SUB_INDUSTRY"]
ret_vars = [f"RET_{i}" for i in range(1,21)]
vol_vars = [f"VOLUME_{i}" for i in range(1,21)]
melt_ret = melt_try.melt(id_vars=keep_vars, value_vars=ret_vars, var_name="TIMESTAMP")
melt_vol = melt_try.melt(id_vars=keep_vars, value_vars=vol_vars, var_name="TIMESTAMP")
melt_ret["TIMESTAMP"] = melt_ret["TIMESTAMP"].str.replace("RET_"," ")
melt_vol["TIMESTAMP"] = melt_ret["TIMESTAMP"].str.replace("VOLUME_"," ")

keep_vars.append("TIMESTAMP")
melt_tot = pd.merge(melt_ret, melt_vol, on=keep_vars)
print(melt_tot.shape)


(8371900, 9)


In [73]:
df = melt_tot.copy()
df = df[["DATE","STOCK","INDUSTRY","SECTOR","TIMESTAMP","value_x","value_y"]]
print(df["INDUSTRY"].value_counts().shape,df["SECTOR"].value_counts().shape)
df = pd.concat([df,pd.get_dummies(df["SECTOR"],prefix="sector_"),pd.get_dummies(df["INDUSTRY"],prefix="industry_")],axis=1)
print(df.shape)
df = df.drop(columns=["INDUSTRY","SECTOR"])
print(df.shape)

(72,) (12,)
(8371900, 91)
(8371900, 89)


In [74]:
df = df[["DATE","STOCK","TIMESTAMP","value_x","value_y"]]

In [75]:
from darts.models.forecasting.transformer_model import TransformerModel
from darts import TimeSeries

model = TransformerModel(
    input_chunk_length=20,
    output_chunk_length=1,
    model_name="transformer"
)

display(df.head())
df = df.fillna(0)
df = df[:10000]

x = TimeSeries.from_group_dataframe(df,group_cols=["DATE","STOCK"],time_col="TIMESTAMP")


,DATE,STOCK,TIMESTAMP,value_x,value_y
0,0,2,1,-0.015748,0.147931
1,0,3,1,0.003984,NaN
2,0,4,1,0.000440,-0.096282
3,0,8,1,0.031298,-0.429540
4,0,14,1,0.027273,-0.847155


In [78]:
model.fit(y_train, x)

KeyError: 0